In [2]:
%pylab
from scipy.stats import spearmanr
from multiprocessing import Pool # this is so we can run in parallel

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [4]:
#load up the data and filter out the stuff we don't want
infall_machs, alphas, mus, sol_fracs, Ngas, seeds, SFEs, Nstars, tenpercent_frac, Mmax_frac, M2_frac = np.loadtxt("SimulationData.dat").T # the .T turns the array sideways so that it lines up with the tuple of variables on the left hand side
infall_machs, alphas, mus, sol_fracs, Ngas, seeds, SFEs, Nstars, tenpercent_frac, Mmax_frac, M2_frac = np.loadtxt("SimulationData.dat")[(alphas>0)*(SFEs>0)*(Mmax_frac<1)].T # the .T turns the array sideways so that it lines up with the tuple of variables on the left hand side

# this is the form of the function you are assuming, which has 2 free parameters X[0] and X[1]
func = lambda X: infall_machs**-2 + X[0]*alphas**2 + X[1]*mus**-2 
# this is the cloud of points in parameter space we will search
grid = np.random.normal(size=(10**6,2)) # 10^6 points sampling from a normal distribution in 2D
grid *= np.array([1e-3,1e-2]) # re-scaling each parameter to have the right order of magnitude (found through experimentation from a wider search)

def GetSpearmanRank(X): # define a function that returns the Spearman rank correlation coefficient
    return spearmanr(func(X), Mmax_frac)[0]

ranks = Pool(16).map(GetSpearmanRank, grid) # run GetSpearmanRank on all elements of grid, on all 16 cores

X = grid[np.abs(ranks).argmax()] # get the parameters that give the strongest correlation

print(grid[np.abs(ranks).argsort()][-10:], np.sort(np.abs(ranks))[-10:]) # print the best 10 parameters - make sure they aren't all over the place, otherwise we might have too much parameter freedom
plt.clf()
plt.scatter(func(X), Mmax_frac,c=np.log10(alphas))


/home/mike/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less
  This is separate from the ipykernel package so we can avoid doing imports until


[[0.00087905 0.01969457]
 [0.0008397  0.0197342 ]
 [0.0008025  0.01963858]
 [0.00080595 0.01981292]
 [0.00080982 0.01970403]
 [0.00079956 0.01969675]
 [0.00078355 0.0172821 ]
 [0.00078139 0.01746481]
 [0.0009835  0.01974849]
 [0.00097882 0.01964341]] [0.92458172 0.92458172 0.92458172 0.92458172 0.92458172 0.92458172
 0.92483912 0.92483912 0.92561133 0.92561133]
